In [1]:
from dataloader import *

In [2]:
import json
config_data = json.load(open('default.json', 'r'))

In [3]:
vocab, train_data_loader, val_data_loader, test_data_loader = get_datasets(config_data)

Using the saved vocab.


/opt/conda/lib/python3.9/site-packages/torchvision/transforms/transforms.py:280: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [4]:
train_data_loader

In [5]:
for i, (images, title, ing, ins, img_id) in enumerate(train_data_loader):
    break

In [6]:
images.shape

torch.Size([64, 3, 256, 256])

In [7]:
title.shape

torch.Size([64, 15])

In [8]:
ing.shape

torch.Size([64, 151])

In [9]:
ins.shape

torch.Size([64, 847])

In [10]:
img_id

('24b0d9e0e0.jpg',
 '263a216a38.jpg',
 '1ce25b8daa.jpg',
 '1a3687d3b7.jpg',
 '64c65e143a.jpg',
 'b9c3384e78.jpg',
 '17b4354f2b.jpg',
 '1c54378de3.jpg',
 '9626c036fb.jpg',
 '090aaba928.jpg',
 'c383251179.jpg',
 'f3eaec2479.jpg',
 '26a8a17d77.jpg',
 'a6a2de8b85.jpg',
 '3adc77f6bc.jpg',
 '1604c7ea34.jpg',
 '04b3e1348c.jpg',
 '454b3c3fb9.jpg',
 '2003cb7314.jpg',
 '130bf469b0.jpg',
 'cdc4253f73.jpg',
 '181267a9a9.jpg',
 '51a6ae2119.jpg',
 '3ddb7cb52f.jpg',
 'e47b0ed2d3.jpg',
 '503a2f1430.jpg',
 'adade13e95.jpg',
 '5b7a87edd2.jpg',
 '5ca2c4772a.jpg',
 'a569a12ba8.jpg',
 'b791bc6585.jpg',
 'e2a920e259.jpg',
 'c99b71785e.jpg',
 '47c058419a.jpg',
 'ab6f57adfe.jpg',
 'abbd3fc587.jpg',
 'b294d6743a.jpg',
 'ee45f0625c.jpg',
 '991ad991bf.jpg',
 'bf53b5e37b.jpg',
 '6991f5acea.jpg',
 '82c18bdeff.jpg',
 '47cb9c3231.jpg',
 'f802d6bd70.jpg',
 'e2cb365537.jpg',
 'b5daea0750.jpg',
 '86a8ac494c.jpg',
 '5492dfc713.jpg',
 '87873c85b4.jpg',
 '670f14b5c1.jpg',
 '1c39f65cdc.jpg',
 'f3dbb408cd.jpg',
 'd09c7276c6

In [12]:
find_img_path(img_id[0])

'~/val/2/4/b/0/24b0d9e0e0.jpg'